# 백테스팅 결과 시각화 가이드

이 노트북은 새로 추가된 백테스팅 결과 시각화 기능들을 사용하는 방법을 보여줍니다.

## 주요 기능

1. **포트폴리오 성과 차트** - 자본 곡선, 벤치마크 비교, 드로다운, 거래 시그널
2. **수익률 분포 히스토그램** - 일간/월간 수익률 분포 분석
3. **월별 수익률 히트맵** - 연도별, 월별 수익률 패턴 분석
4. **성과 비교 표** - 전략 vs 벤치마크 종합 지표 비교


In [1]:
# 프로젝트 루트를 Python 경로에 추가
import sys
import os
from pathlib import Path

# 현재 노트북의 위치에서 프로젝트 루트 찾기
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # examples 폴더에서 실행하는 경우
    project_root = current_dir.parent.parent
else:
    # 프로젝트 루트에서 실행하는 경우
    project_root = current_dir

# 프로젝트 루트를 Python 경로에 추가
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 필요한 라이브러리 import
import asyncio
import polars as pl
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# quantbt 라이브러리
from typing import List

# quantbt 라이브러리 import
from quantbt import (
    UpbitDataProvider,
    SimpleBacktestEngine,
    SimpleBroker,
    TradingStrategy,
    MarketDataBatch,
    BacktestConfig,
    Order,
    OrderType,
    OrderSide,
)

# 시각화 라이브러리
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

print("✅ 라이브러리 import 완료")


✅ 라이브러리 import 완료


In [2]:
# 간단한 매수후보유 전략 예제
class BuyAndHoldStrategy(TradingStrategy):
    """매수후보유 전략 (시각화 테스트용)"""
    
    def __init__(self):
        super().__init__(
            name="BuyAndHoldStrategy",
            config={},
            position_size_pct=0.8,  # 80%로 포지션
            max_positions=10
        )
        self.bought_symbols: set[str] = set()  # 이미 매수한 심볼들 추적
        
    def _compute_indicators_for_symbol(self, symbol_data: pl.DataFrame) -> pl.DataFrame:
        """바이 앤 홀드는 지표 불필요 - 원본 데이터 그대로 반환"""
        return symbol_data
    
    def generate_signals(self, data: MarketDataBatch) -> List[Order]:
        """신호 생성 - 한 번만 매수"""
        orders = []
        
        if not self.context:
            return orders
        
        # 아직 매수하지 않은 심볼 중 선택하여 매수
        for symbol in data.symbols:
            if symbol not in self.bought_symbols:
                current_price = self.get_current_price(symbol, data)
                
                if current_price and current_price > 0:
                    # 현재 포트폴리오 가치의 일정 비율로 매수
                    portfolio_value = self.get_portfolio_value()
                    position_value = portfolio_value / len(self.context.symbols) * 0.4  # 40%만 사용
                    quantity = position_value / current_price
                    
                    # 최소 수량 확인
                    if quantity > 0.01:
                        order = Order(
                            symbol=symbol,
                            side=OrderSide.BUY,
                            quantity=quantity,
                            order_type=OrderType.MARKET
                        )
                        orders.append(order)
                        self.bought_symbols.add(symbol)
        
        return orders

In [3]:
# 1. 업비트 데이터 프로바이더
upbit_provider = UpbitDataProvider()

In [4]:
# 2. 백테스팅 설정 (2024년 1년)
config = BacktestConfig(
    symbols=["KRW-BTC"],
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31),
    timeframe="1d",  # 일봉
    initial_cash=10_000_000,  # 1천만원
    commission_rate=0.0,      # 수수료 0% (테스트용) - 실제 백테스팅에는 적절한 값 사용
    slippage_rate=0.0,         # 슬리피지 0% (테스트용) - 실제 백테스팅에는 적절한 값 사용
    visualization_mode=True,  # 시각화 모드 활성화
    save_trades=True,
    save_portfolio_history=True
)


In [5]:
# 3. 간단한 매수후보유 전략 예제
strategy = BuyAndHoldStrategy()

In [6]:
# 4. 브로커 설정
broker = SimpleBroker(
    initial_cash=config.initial_cash,
    commission_rate=config.commission_rate,
    slippage_rate=config.slippage_rate
)

In [7]:
# 5. 백테스트 엔진
engine = SimpleBacktestEngine()
engine.set_strategy(strategy)
engine.set_data_provider(upbit_provider)
engine.set_broker(broker)

In [8]:
# 6. 백테스팅 실행
result = await engine.run(config)

  💾 캐시에서 데이터 로드: KRW-BTC 1d (365개 캔들)
    📅 캐시 범위: 2024-01-01 ~ 2024-12-30
    📊 커버리지: 99.7% (365/366일)


In [9]:

if result.config.visualization_mode:
    print("✅ 시각화 모드가 활성화되어 있습니다.")
    
    # 시각화 데이터 확인
    print(f"- equity_curve: {'있음' if result.equity_curve is not None else '없음'}")
    print(f"- daily_returns: {'있음' if result.daily_returns is not None else '없음'}")
    print(f"- monthly_returns: {'있음' if result.monthly_returns is not None else '없음'}")
    print(f"- benchmark_equity_curve: {'있음' if result.benchmark_equity_curve is not None else '없음'}")
    print(f"- drawdown_periods: {'있음' if result.drawdown_periods is not None else '없음'}")
    print(f"- trade_signals: {'있음' if result.trade_signals is not None else '없음'}")
    
    print("\n주피터 노트북에서 사용 가능한 시각화 메서드:")
    print("1. result.plot_portfolio_performance() - 포트폴리오 성과 차트")
    print("2. result.plot_returns_distribution() - 수익률 분포 히스토그램")
    print("3. result.plot_monthly_returns_heatmap() - 월별 수익률 히트맵")
    print("4. result.show_performance_comparison() - 벤치마크 비교 표")
    
    # 샘플 호출 (주피터 노트북 환경이 아니므로 실제 차트는 생성되지 않음)
    try:
        print("\n시각화 메서드 호출 테스트...")
        result.plot_portfolio_performance()
        result.plot_returns_distribution(period="daily")
        result.plot_monthly_returns_heatmap()
        result.show_performance_comparison()
    except Exception as e:
        print(f"시각화 테스트 중 예상된 오류 (plotly 없음): {e}")

else:
    print("❌ 시각화 모드가 비활성화되어 있습니다.")

✅ 시각화 모드가 활성화되어 있습니다.
- equity_curve: 있음
- daily_returns: 있음
- monthly_returns: 있음
- benchmark_equity_curve: 있음
- drawdown_periods: 있음
- trade_signals: 있음

주피터 노트북에서 사용 가능한 시각화 메서드:
1. result.plot_portfolio_performance() - 포트폴리오 성과 차트
2. result.plot_returns_distribution() - 수익률 분포 히스토그램
3. result.plot_monthly_returns_heatmap() - 월별 수익률 히트맵
4. result.show_performance_comparison() - 벤치마크 비교 표

시각화 메서드 호출 테스트...


,지표,전략,벤치마크
0,총 수익률 (%),107.48,7.00
1,연간 수익률 (%),107.59,4.78
2,변동성 (%),15.00,0.00
3,샤프 비율,6.97,26501811059197.94
4,칼마 비율,10.76,0.00
5,소르티노 비율,9.55,0.00
6,최대 낙폭 (%),10.00,0.00
7,베타,0.00,1.00
8,알파,1.08,0.00
9,총 거래 횟수,1,-


In [ ]:

result.print_summary()